# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457774 , -1.8541754 , -5.1695833 ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884269 ],
       [ 4.6476874 ,  8.377879  , -9.070581  ,  9.459332  ,  8.450423  ,
        -1.0210547 ,  3.392087  , -7.8629856 , -0.7527662 ,  0.48384118],
       [-2.9414437 ,  4.6401706 , -4.5027537 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.816936  ],
       [-4.271077  ,  5.561165  , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.5837417 , -4.1515303 ,  4.369667  , -3.0020504 ,  3.638897  ,
        -4.341912  , -3.318711  ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:42962 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809874  ,  8.422671  , -9.239023  ,  9.379143  ,  8.499881  ,
        -1.0592818 ,  3.3437858 , -7.802612  , -0.5946333 ,  0.26447597,
         5.5073957 , -4.1069803 ,  4.2890778 , -2.8172054 ,  3.6150153 ,
        -4.1613    , -3.6209643 ,  6.218529  , -6.946047  , -1.0828307 ,
        -5.82677   ,  2.225876  , -3.8601215 , -1.6974077 , -5.3134174 ,
         7.579578  ,  2.9187472 ,  8.540423  ,  1.5523205 ,  1.0841805 ],
       [-2.8941853 ,  4.4741907 , -4.4475675 ,  2.3820987 ,  1.7478832 ,
        -2.5046248 , -5.2083306 , -1.6937687 , -8.134755  ,  2.6468296 ,
        -4.316362  ,  5.56554   , -5.732198  , -1.7384952 , -9.344658  ,
         0.7084658 ,  4.4358397 , -2.9009    , -4.9486375 ,  9.695301  ,
         8.366521  , -6.2474537 , -6.3494725 ,  1.9546973 ,  4.157616  ,
        -9.167902  ,  4.6070676 ,  8.788584  ,  6.864423  ,  2.2319882 ],
       [-4.665713  , -9.558958  ,  6.6572285 ,  4.440131  ,  2.1730306 ,
         2.5904036 ,  0.58000994,  6.255035  , -8